# Kinova Gen3 Kinematic and Dynamic Analysis

In [1]:
import os
from pathlib import Path
from sympy import diff, Matrix, pi, symbols, Symbol
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing, Point, ReferenceFrame, \
    RigidBody, inertia, KanesMethod
from function import PythonFunction
from function import CppFunction
from function import JuliaFunction

In [2]:
mechanics_printing(pretty_print=True)

Select file type for function generation, if True generate function for Python, else generate for C++

In [3]:
generate_output_file_for_python = False
generate_output_file_for_cpp = True
generate_output_file_for_julia = False

Path(os.path.join('.', 'src')).mkdir(exist_ok=True)

if generate_output_file_for_python:
    # Output folder
    src_folder_path_python = os.path.join('.', 'src', 'python')
    Path(src_folder_path_python).mkdir(exist_ok=True)

if generate_output_file_for_cpp:
    # Output folder
    src_folder_path_cpp = os.path.join('.', 'src', 'cpp')
    Path(src_folder_path_cpp).mkdir(exist_ok=True)

if generate_output_file_for_julia:
    # Output folder
    src_folder_path_julia = os.path.join('.', 'src', 'julia')
    Path(src_folder_path_julia).mkdir(exist_ok=True)

Set to ```True``` if using Robotiq 2f-85 gripper at the end-effector, ```False``` otherwise

In [4]:
gripper = False

Define kinematic variables for the analysis. For $i\in[1, 7]$, $q_i$ are joint positions, $q_{i}p$ are joint velocities, $u_i$ and $u_{i}p$ are generalized speeds and their derivatives.

In [5]:
q1, q2, q3, q4, q5, q6, q7 = dynamicsymbols("q1 q2 q3 q4 q5 q6 q7")
q1p, q2p, q3p, q4p, q5p, q6p, q7p = dynamicsymbols("q1 q2 q3 q4 q5 q6 q7", 1)
u1, u2, u3, u4, u5, u6, u7 = dynamicsymbols("u1 u2 u3 u4 u5 u6 u7")
u1p, u2p, u3p, u4p, u5p, u6p, u7p = dynamicsymbols("u1 u2 u3 u4 u5 u6 u7", 1)

In [6]:
# Lists of generalized coordinates and speeds
q = [q1, q2, q3, q4, q5, q6, q7]
qp = [q1p, q2p, q3p, q4p, q5p, q6p, q7p]
u = [u1, u2, u3, u4, u5, u6, u7]

Define the joint torques

In [7]:
TA, TB, TC, TD, TE, TF, TG = symbols("TA TB TC TD TE TF TG")

Define the external forces and moments acting on the end-effector of the robot

In [8]:
# Forces and moments acting on the end-effector
Fx, Fy, Fz = symbols('Fx Fy Fz')
Mx, My, Mz = symbols('Mx My Mz')

Gravitational constant is g

In [9]:
g = symbols('g')

Create dummy variables for prettier printing of kinematic and dynamic equations

In [10]:
dummy_dict = dict(zip(q + qp + u,
                      ["q1", "q2", "q3", "q4", "q5", "q6", "q7"] +
                      ["qp1", "qp2", "qp3", "qp4", "qp5", "qp6", "qp7"] +
                      ["u1", "u2", "u3", "u4", "u5", "u6", "u7"]))

Define the reference frames where **N** is the Newtonian reference frame and **A**, **B**, **C**, **D**, **E**, **F**, **G** are the reference frames assigned to each link. **H** is the last reference frame attached to the end-effector. It is also used as the reference frame of the gripper if it is used.

In [11]:
N = ReferenceFrame("N")
A = N.orientnew("A", "Body", [pi, 0, q1], "123")
B = A.orientnew("B", "Body", [pi / 2, 0, q2], "123")
C = B.orientnew("C", "Body", [-pi / 2, 0, q3], "123")
D = C.orientnew("D", "Body", [pi / 2, 0, q4], "123")
E = D.orientnew("E", "Body", [-pi / 2, 0, q5], "123")
F = E.orientnew("F", "Body", [pi / 2, 0, q6], "123")
G = F.orientnew("G", "Body", [-pi / 2, 0, q7], "123")
H = G.orientnew("H", "Body", [pi, 0, 0], "123")

## Kinematic Analysis

### Position-level kinematics

Define the kinematic structure by assigning stationary points for each reference frame.

- *O* is fixed in **N**.
- *$P_1$* is fixed in **A**.
- *$P_2$* is fixed in **B**.
- *$P_3$* is fixed in **C**.
- *$P_4$* is fixed in **D**.
- *$P_5$* is fixed in **E**.
- *$P_6$* is fixed in **F**.
- *$P_7$* is fixed in **G**.
- *$P_8$* is fixed in **H**.
- *$P_9$* is fixed in **H**.

In [12]:
P0 = Point("O")
P1 = P0.locatenew("P1", 0.15643 * N.z)
P2 = P1.locatenew("P2", 0.005375 * A.y - 0.12838 * A.z)
P3 = P2.locatenew("P3", -0.21038 * B.y - 0.006375 * B.z)
P4 = P3.locatenew("P4", 0.006375 * C.y - 0.21038 * C.z)
P5 = P4.locatenew("P5", -0.20843 * D.y - 0.006375 * D.z)
P6 = P5.locatenew("P6", 0.00017505 * E.y - 0.10593 * E.z)
P7 = P6.locatenew("P7", -0.10593 * F.y - 0.00017505 * F.z)
P8 = P7.locatenew("P8", -0.0615 * G.z)

if gripper:
    P9 = P8.locatenew("P9", 0.12 * H.z)

Define the location of mass centres for each link.

- *$A_o$* is the mass center for body <u>A</u> defined in reference frame **A**.
- *$B_o$* is the mass center for body <u>B</u> defined in reference frame **B**.
- *$C_o$* is the mass center for body <u>C</u> defined in reference frame **C**.
- *$D_o$* is the mass center for body <u>D</u> defined in reference frame **D**.
- *$E_o$* is the mass center for body <u>E</u> defined in reference frame **E**.
- *$F_o$* is the mass center for body <u>F</u> defined in reference frame **F**.
- *$G_o$* is the mass center for body <u>G</u> defined in reference frame **G**.
- *$H_o$* is the mass center for body <u>H</u>, the gripper, defined in reference frame **H**.

In [13]:
Ao = P1.locatenew("Ao", -0.000023 * A.x - 0.010364 * A.y - 0.073360 * A.z)
Bo = P2.locatenew("Bo", -0.000044 * B.x - 0.099580 * B.y - 0.013278 * B.z)
Co = P3.locatenew("Co", -0.000044 * C.x - 0.006641 * C.y - 0.117892 * C.z)
Do = P4.locatenew("Do", -0.000018 * D.x - 0.075478 * D.y - 0.015006 * D.z)
Eo = P5.locatenew("Eo", 0.000001 * E.x - 0.009432 * E.y - 0.063883 * E.z)
Fo = P6.locatenew("Fo", 0.000001 * F.x - 0.045483 * F.y - 0.009650 * F.z)
Go = P7.locatenew("Go", -0.000281 * G.x - 0.011402 * G.y - 0.029798 * G.z)

if gripper:
    Ho = P8.locatenew("Ho", 0.058 * H.z)

Calculate the end-effector position and orientation of the robot.

In [14]:
x = P9.pos_from(P0).express(N).subs(dummy_dict).to_matrix(N) if gripper \
        else P8.pos_from(P0).express(N).subs(dummy_dict).to_matrix(N)

R = N.dcm(H).subs(dummy_dict)

Print end-effector position and rotation functions

### Velocity-level kinematics

Calculate the valocities of the points defined in the position-level kinematics.

In [15]:
P0.set_vel(N, 0 * N.x + 0 * N.y + 0 * N.z)
P1.v2pt_theory(P0, N, N)
P2.v2pt_theory(P1, N, A)
P3.v2pt_theory(P2, N, B)
P4.v2pt_theory(P3, N, C)
P5.v2pt_theory(P4, N, D)
P6.v2pt_theory(P5, N, E)
P7.v2pt_theory(P6, N, F)
P8.v2pt_theory(P7, N, G)

if gripper:
    P9.v2pt_theory(P8, N, H)

Ao.v2pt_theory(P1, N, A)
Bo.v2pt_theory(P2, N, B)
Co.v2pt_theory(P3, N, C)
Do.v2pt_theory(P4, N, D)
Eo.v2pt_theory(P5, N, E)
Fo.v2pt_theory(P6, N, F)
Go.v2pt_theory(P7, N, G)

if gripper:
    Ho.v2pt_theory(P8, N, H)

Define the kinematic differential equations.

In [16]:
kd = [qp_i - u_i for (qp_i, u_i) in zip(qp, u)]

Calculate the translational and the rotational part of the Jacobian.

In [17]:
Jt = Matrix([P9.pos_from(P0).dot(N.x),
             P9.pos_from(P0).dot(N.y),
             P9.pos_from(P0).dot(N.z)])\
        .jacobian(Matrix([q1, q2, q3, q4, q5, q6, q7])) if gripper \
        else Matrix([P8.pos_from(P0).dot(N.x),
                     P8.pos_from(P0).dot(N.y),
                     P8.pos_from(P0).dot(N.z)])\
        .jacobian(Matrix([q1, q2, q3, q4, q5, q6, q7]))

Jr = Matrix(
            [
                [
                    H.ang_vel_in(N).to_matrix(N)[0].coeff(q1p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[0].coeff(q2p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[0].coeff(q3p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[0].coeff(q4p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[0].coeff(q5p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[0].coeff(q6p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[0].coeff(q7p).trigsimp(),
                ],
                [
                    H.ang_vel_in(N).to_matrix(N)[1].coeff(q1p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[1].coeff(q2p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[1].coeff(q3p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[1].coeff(q4p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[1].coeff(q5p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[1].coeff(q6p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[1].coeff(q7p).trigsimp(),
                ],
                [
                    H.ang_vel_in(N).to_matrix(N)[2].coeff(q1p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[2].coeff(q2p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[2].coeff(q3p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[2].coeff(q4p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[2].coeff(q5p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[2].coeff(q6p).trigsimp(),
                    H.ang_vel_in(N).to_matrix(N)[2].coeff(q7p).trigsimp(),
                ],
            ]
           )

J = Jt.col_join(Jr)
J_dot = J.diff()
J_subs = J.subs(dummy_dict)
J_dot_subs = J_dot.subs(dummy_dict)

## Dynamic Analysis

Define the masses of the links and the gripper

- $m_A$ is the mass of the body **A**
- $m_B$ is the mass of the body **B**
- $m_C$ is the mass of the body **C**
- $m_D$ is the mass of the body **D**
- $m_E$ is the mass of the body **E**
- $m_F$ is the mass of the body **F**
- $m_G$ is the mass of the body **G**
- $m_H$ is the mass of the gripper, aka body **H**

In [18]:
mass_A = 1.3773
mass_B = 1.1636
mass_C = 1.1636
mass_D = 0.9302
mass_E = 0.6781
mass_F = 0.6781
mass_G = 0.5006

if gripper:
    mass_H = 0.925

Define the principal inertia for each body with respect to the body's center of mass

In [19]:
inertia_A = inertia(A, 0.004570, 0.004831, 0.001409, 0.000001, 0.000448, 0.000002)
inertia_B = inertia(B, 0.011088, 0.001072, 0.011255, 0.000005, -0.000691, 0.000000)
inertia_C = inertia(C, 0.010932, 0.011127, 0.001043, 0.000000, 0.000606, -0.000007)
inertia_D = inertia(D, 0.008147, 0.000631, 0.008316, -0.000001, -0.000500, 0.000000)
inertia_E = inertia(E, 0.001596, 0.001607, 0.000399, 0.000000, 0.000256, 0.000000)
inertia_F = inertia(F, 0.001641, 0.000410, 0.001641, 0.000000, -0.000278, 0.000000)
inertia_G = inertia(G, 0.000587, 0.000369, 0.000609, 0.000003, 0.000118, 0.000003)

# Inertia of the gripper
if gripper:
    inertia_H = inertia(H, 4180e-6, 5080e-6, 1250e-6)

Define the rigid bodies for the dynamic analysis

In [20]:
body_A = RigidBody('body_A', Ao, A, mass_A, (inertia_A, Ao))
body_B = RigidBody('body_B', Bo, B, mass_B, (inertia_B, Bo))
body_C = RigidBody('body_C', Co, C, mass_C, (inertia_C, Co))
body_D = RigidBody('body_D', Do, D, mass_D, (inertia_D, Do))
body_E = RigidBody('body_E', Eo, E, mass_E, (inertia_E, Eo))
body_F = RigidBody('body_F', Fo, F, mass_F, (inertia_F, Fo))
body_G = RigidBody('body_G', Go, G, mass_G, (inertia_G, Go))

if gripper:
    body_H = RigidBody('body_H', Ho, H, mass_H, (inertia_H, Ho))

body_list = [body_A, body_B, body_C, body_D, body_E, body_F, body_G, body_H] if gripper else [body_A, body_B, body_C, body_D, body_E, body_F, body_G]

Define the forces acting on the rigid bodies.

In [21]:
force_list = [(Ao, -mass_A * g * N.z),
              (Bo, -mass_B * g * N.z),
              (Co, -mass_C * g * N.z),
              (Do, -mass_D * g * N.z),
              (Eo, -mass_E * g * N.z),
              (Fo, -mass_F * g * N.z),
              (Go, -mass_G * g * N.z),
              (Ho, -mass_H * g * N.z),
              (A, TA * A.z),
              (B, TB * B.z),
              (C, TC * C.z),
              (D, TD * D.z),
              (E, TE * E.z),
              (F, TF * F.z),
              (G, TG * G.z),
              (P9, Fx * N.x + Fy * N.y + Fz * N.z)] if gripper \
        else [(Ao, -mass_A * g * N.z),
              (Bo, -mass_B * g * N.z),
              (Co, -mass_C * g * N.z),
              (Do, -mass_D * g * N.z),
              (Eo, -mass_E * g * N.z),
              (Fo, -mass_F * g * N.z),
              (Go, -mass_G * g * N.z),
              (A, TA * A.z),
              (B, TB * B.z),
              (C, TC * C.z),
              (D, TD * D.z),
              (E, TE * E.z),
              (F, TF * F.z),
              (G, TG * G.z),
              (P8, Fx * N.x + Fy * N.y + Fz * N.z)]

Use Kane's method to solve for the equations of motion.

In [22]:
KM = KanesMethod(N, q_ind=q, u_ind=u, kd_eqs=kd)

(fr, frstar) = KM.kanes_equations(body_list, force_list)

Get the mass matrix of the robot

In [23]:
M = KM.mass_matrix.subs(dummy_dict)

Coriolis and gravity terms

In [24]:
forcing = -KM.forcing.subs(dummy_dict)

In [25]:
# Gravity term
G = Matrix([forcing[0].coeff(g) * g,
            forcing[1].coeff(g) * g,
            forcing[2].coeff(g) * g,
            forcing[3].coeff(g) * g,
            forcing[4].coeff(g) * g,
            forcing[5].coeff(g) * g,
            forcing[6].coeff(g) * g])

# Coriolis term
C = forcing.subs({TA: 0, TB: 0, TC: 0, TD: 0, TE: 0, TF: 0, TG: 0, g: 0,
                  Fx: 0, Fy: 0, Fz: 0})

## Printing

### Kinematics functions

Print forward kinematics $\phi(q)$

In [26]:
if generate_output_file_for_python:
    forward_kinematics_printer = PythonFunction('forward_kinematics', src_folder_path_python, ['q'], ['x', 'R'], [x, R])
    forward_kinematics_printer.print()

In [27]:
if generate_output_file_for_cpp:
    forward_kinematics_printer_cpp = CppFunction('forwardKinematics', src_folder_path_cpp, {'q':'Eigen::Vector<double, 7>'}, {'T': 'Eigen::Transform<double, 3, Eigen::Affine>'}, {'x': 'Eigen::Vector<double, 3>', 'R': 'Eigen::Matrix<double, 3, 3>'}, [x, R])
    forward_kinematics_printer_cpp.print()

Printing the function: forwardKinematics


In [28]:
if generate_output_file_for_julia:
    forward_kinematics_printer_julia = JuliaFunction('forward_kinematics', src_folder_path_julia, {'q':'SVector{7, Float64}'}, {'x': 'SVector{3, Float64}', 'R': 'SMatrix{3, 3, Float64}'}, [x, R])
    forward_kinematics_printer_julia.print()

Print Jacobian and it derivative $J(q)$, $\dot{J}(q, \dot{q})$ 

In [29]:
if generate_output_file_for_python:
    jacobian_printer = PythonFunction('jacobian', src_folder_path_python, ['q'], ['J'], J_subs)
    jacobian_dot_printer = PythonFunction('jacobian_dot', src_folder_path_python, ['q', 'qp'], ['Jdot'], J_dot_subs)
    jacobian_printer.print()
    jacobian_dot_printer.print()

In [30]:
if generate_output_file_for_cpp:
    jacobian_printer_cpp = CppFunction('jacobian', src_folder_path_cpp, {'q':'Eigen::Vector<double, 7>'}, {'J': 'Eigen::Matrix<double, 6, 7>'}, set(), J_subs)
    jacobian_dot_printer_cpp = CppFunction('jacobian_dot', src_folder_path_cpp, {'q':'Eigen::Vector<double, 7>', 'qp':'Eigen::Vector<double, 7>'}, {'Jdot': 'Eigen::Matrix<double, 6, 7>'}, set(), J_dot_subs)
    jacobian_printer_cpp.print()
    jacobian_dot_printer_cpp.print()

Printing the function: jacobian
Printing the function: jacobian_dot


In [31]:
if generate_output_file_for_julia:
    jacobian_printer_julia = JuliaFunction('jacobian', src_folder_path_julia, {'q':'SVector{7, Float64}'}, {'J': 'SMatrix{6, 7, Float64}>'}, J_subs)
    jacobian_dot_printer_julia = JuliaFunction('jacobian_dot', src_folder_path_julia, {'q':'SVector{7, Float64}', 'qp':'SVector{7, Float64}'}, {'J_dot': 'SMatrix{6, 7, Float64}>'}, J_dot_subs)
    jacobian_printer_julia.print()
    jacobian_dot_printer_julia.print()

### Dynamics functions

Print mass matrix $M(q)$

In [32]:
if generate_output_file_for_python:
    mass_matrix_printer_py = PythonFunction('mass_matrix', src_folder_path_python, ['q'], ['M'], M)
    mass_matrix_printer_py.print()

In [33]:
if generate_output_file_for_cpp:
    mass_matrix_printer_cpp = CppFunction('mass_matrix', src_folder_path_cpp, {'q':'Eigen::Vector<double, 7>'}, {'M': 'Eigen::Matrix<double, 7, 7>'}, set(), M)
    mass_matrix_printer_cpp.print()

Printing the function: mass_matrix


In [34]:
if generate_output_file_for_julia:
    mass_matrix_printer_julia = JuliaFunction('mass_matrix', src_folder_path_julia, {'q':'SVector{7, Float64}'}, {'M':'SMatrix{7, 7, Float64}'}, M)
    mass_matrix_printer_julia.print()

Print gravity term $g(q)$

In [35]:
if generate_output_file_for_python:
    gravity_term_printer = PythonFunction('gravity', src_folder_path_python, ['q'], ['G'], G)
    gravity_term_printer.print()

In [36]:
if generate_output_file_for_cpp:
    gravity_term_printer_cpp = CppFunction('gravity', src_folder_path_cpp, {'q': 'Eigen::Vector<double, 7>'}, {'G': 'Eigen::Vector<double, 7>'}, set(), G)
    gravity_term_printer_cpp.print()

Printing the function: gravity


In [37]:
if generate_output_file_for_julia:
    gravity_term_printer_julia = JuliaFunction('gravity', src_folder_path_julia, {'q':'SVector{7, Float64}'}, {'G':'SVector{7, Float64}'}, G)
    gravity_term_printer_julia.print()

Print Coriolis term $C(q, \dot{q})\dot{q}$

In [38]:
if generate_output_file_for_python:
    gravity_term_printer = PythonFunction('coriolis', src_folder_path_python, ['q', 'qp'], ['C'], C)
    gravity_term_printer.print()

In [39]:
if generate_output_file_for_cpp:
    gravity_term_printer_julia = CppFunction('coriolis', src_folder_path_cpp, {'q': 'Eigen::Vector<double, 7>', 'qp': 'Eigen::Vector<double, 7>'}, {'C':'Eigen::Vector<double, 7>'}, set(), C)
    gravity_term_printer_julia.print()

Printing the function: coriolis


In [40]:
if generate_output_file_for_julia:
    gravity_term_printer_julia = JuliaFunction('coriolis', src_folder_path_julia, {'q': 'SVector{7, Float64}', 'qp': 'SVector{7, Float64}'}, {'C': 'SVector{7, Float64}'}, C)
    gravity_term_printer_julia.print()